In [ ]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (147 kB/s)
Reading package lists... Done


In [ ]:
from pyspark import SparkContext, SparkConf
#Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final Project Analysis").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Read in data from S3 Buckets
url20 = "https://cps-final-project-bucket.s3.us-east-2.amazonaws.com/metrics_collenrollpersist_schoollevel_20222_CLEAN_3+(2).csv"
spark.sparkContext.addFile(url20)
persist_3_df = spark.read.csv(SparkFiles.get("metrics_collenrollpersist_schoollevel_20222_CLEAN_3+(2).csv"), sep=",", header=True)

persist_3_df.show()

+---------+----------------------+------+-----------------------+-------------------------+----------------------------+-----------------------------------------+-----------------------------+-----------------------+-------------------------+----------------------------+-----------------------------------------+-----------------------------+-----------------------+-------------------------+----------------------------+-----------------------------------------+-----------------------------+-----------------------+-------------------------+----------------------------+-----------------------------------------+-----------------------------+-----------------------+-------------------------+----------------------------+-----------------------------------------+-----------------------------+
|School_ID|Annualized School Name|status|Class of 2019 Graduates|Class of 2019 Enrollments|Class of 2019 Enrollment Pct|Class of 2019 # of Enrollments Persisting|Class of 2019 Persistence Pct|Class of 20

In [ ]:
persist_3_df = persist_3_df.toPandas()
persist_3_df.head()

,School_ID,Annualized School Name,status,Class of 2019 Graduates,Class of 2019 Enrollments,Class of 2019 Enrollment Pct,Class of 2019 # of Enrollments Persisting,Class of 2019 Persistence Pct,Class of 2018 Graduates,Class of 2018 Enrollments,...,Class of 2016 Graduates,Class of 2016 Enrollments,Class of 2016 Enrollment Pct,Class of 2016 # of Enrollments Persisting,Class of 2016 Persistence Pct,Class of 2015 Graduates,Class of 2015 Enrollments,Class of 2015 Enrollment Pct,Class of 2015 # of Enrollments Persisting,Class of 2015 Persistence Pct
0,400013,ASPIRA - EARLY COLLEGE HS,None,75,30,40,18,60,83,49,...,103,57,55.3,37,64.9,78,23,29.5,13,56.5
1,400022,CHIARTS HS,None,133,114,85.7,97,85.1,146,117,...,112,90,80.4,79,87.8,113,90,79.6,80,88.9
2,400032,CICS - ELLISON HS,None,101,62,61.4,31,50,72,47,...,89,45,50.6,28,62.2,98,63,64.3,42,66.7
3,400033,CICS - LONGWOOD,None,87,44,50.6,30,68.2,107,52,...,87,39,44.8,28,71.8,77,50,64.9,28,56
4,400034,CICS - NORTHTOWN HS,None,207,151,72.9,114,75.5,178,145,...,176,132,75,101,76.5,167,128,76.6,108,84.4


In [ ]:
persist_3_df.rename(columns={"School_ID": "School ID"}, inplace=True)
persist_3_df

,School ID,Annualized School Name,status,Class of 2019 Graduates,Class of 2019 Enrollments,Class of 2019 Enrollment Pct,Class of 2019 # of Enrollments Persisting,Class of 2019 Persistence Pct,Class of 2018 Graduates,Class of 2018 Enrollments,...,Class of 2016 Graduates,Class of 2016 Enrollments,Class of 2016 Enrollment Pct,Class of 2016 # of Enrollments Persisting,Class of 2016 Persistence Pct,Class of 2015 Graduates,Class of 2015 Enrollments,Class of 2015 Enrollment Pct,Class of 2015 # of Enrollments Persisting,Class of 2015 Persistence Pct
0,400013,ASPIRA - EARLY COLLEGE HS,None,75,30,40,18,60,83,49,...,103,57,55.3,37,64.9,78,23,29.5,13,56.5
1,400022,CHIARTS HS,None,133,114,85.7,97,85.1,146,117,...,112,90,80.4,79,87.8,113,90,79.6,80,88.9
2,400032,CICS - ELLISON HS,None,101,62,61.4,31,50,72,47,...,89,45,50.6,28,62.2,98,63,64.3,42,66.7
3,400033,CICS - LONGWOOD,None,87,44,50.6,30,68.2,107,52,...,87,39,44.8,28,71.8,77,50,64.9,28,56
4,400034,CICS - NORTHTOWN HS,None,207,151,72.9,114,75.5,178,145,...,176,132,75,101,76.5,167,128,76.6,108,84.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,610570,OMBUDSMAN - SOUTH HS,None,88,16,18.2,5,31.3,95,21,...,89,21,23.6,9,42.9,39,5,12.8,None,None
167,610571,OMBUDSMAN - WEST HS,None,54,16,29.6,11,68.8,44,6,...,70,11,15.7,3,27.3,54,4,7.4,None,None
168,610587,DYETT ARTS HS,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
169,610593,OMBUDSMAN - ROSELAND HS,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
persist_3_df = persist_3_df.set_index(['School ID'])
persist_3_df

,Annualized School Name,status,Class of 2019 Graduates,Class of 2019 Enrollments,Class of 2019 Enrollment Pct,Class of 2019 # of Enrollments Persisting,Class of 2019 Persistence Pct,Class of 2018 Graduates,Class of 2018 Enrollments,Class of 2018 Enrollment Pct,...,Class of 2016 Graduates,Class of 2016 Enrollments,Class of 2016 Enrollment Pct,Class of 2016 # of Enrollments Persisting,Class of 2016 Persistence Pct,Class of 2015 Graduates,Class of 2015 Enrollments,Class of 2015 Enrollment Pct,Class of 2015 # of Enrollments Persisting,Class of 2015 Persistence Pct
School ID,,,,,,,,,,,,,,,,,,,,,
400013,ASPIRA - EARLY COLLEGE HS,None,75,30,40,18,60,83,49,59,...,103,57,55.3,37,64.9,78,23,29.5,13,56.5
400022,CHIARTS HS,None,133,114,85.7,97,85.1,146,117,80.1,...,112,90,80.4,79,87.8,113,90,79.6,80,88.9
400032,CICS - ELLISON HS,None,101,62,61.4,31,50,72,47,65.3,...,89,45,50.6,28,62.2,98,63,64.3,42,66.7
400033,CICS - LONGWOOD,None,87,44,50.6,30,68.2,107,52,48.6,...,87,39,44.8,28,71.8,77,50,64.9,28,56
400034,CICS - NORTHTOWN HS,None,207,151,72.9,114,75.5,178,145,81.5,...,176,132,75,101,76.5,167,128,76.6,108,84.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610570,OMBUDSMAN - SOUTH HS,None,88,16,18.2,5,31.3,95,21,22.1,...,89,21,23.6,9,42.9,39,5,12.8,None,None
610571,OMBUDSMAN - WEST HS,None,54,16,29.6,11,68.8,44,6,13.6,...,70,11,15.7,3,27.3,54,4,7.4,None,None
610587,DYETT ARTS HS,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
url21 = "https://cps-final-project-bucket.s3.us-east-2.amazonaws.com/metrics_cohortgraduationdropoutadjusted_schoollevel_2011to2019.csv"
spark.sparkContext.addFile(url21)
grad_and_dropout_df = spark.read.csv(SparkFiles.get("metrics_cohortgraduationdropoutadjusted_schoollevel_2011to2019.csv"), sep=",", header=True)

In [ ]:
grad_and_dropout_df = grad_and_dropout_df.toPandas()
grad_and_dropout_df.head()

,School ID,School Name,Status as of 2019,2011 Grad. %,2012 Grad. %,2013 Grad. %,2014 Grad. %,2015 Grad. %,2016 Grad. %,2017 Grad. %,2018 Grad. %,2019 Grad. %
0,400121,ACERO - DE LA CRUZ,None,,,,,,,,78.1,79.5
1,400085,ACERO - GARCIA HS,None,,,75.5,85.1,89.5,84.5,89.2,86.3,87.3
2,400149,ACERO - SOTO HS,None,,,,,,,,83.9,71.4
3,610513,AIR FORCE HS,None,,,,70.2,84.0,86.1,86.7,81.9,83.7
4,610524,ALCOTT HS,None,,,,86.3,77.7,87.1,81.3,84.9,92.1


In [ ]:
url22 = "https://cps-final-project-bucket.s3.us-east-2.amazonaws.com/metrics_cohortgraduationdropoutadjusted_schoollevel_2011to2019+(1).csv"
spark.sparkContext.addFile(url22)
grad_and_dropout_2_df = spark.read.csv(SparkFiles.get("metrics_cohortgraduationdropoutadjusted_schoollevel_2011to2019+(1).csv"), sep=",", header=True)

grad_and_dropout_2_df.show()

+---------+--------------------+-----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|School ID|         School Name|Status as of 2019|2011 Grad. %|2012 Grad. %|2013 Grad. %|2014 Grad. %|2015 Grad. %|2016 Grad. %|2017 Grad. %|2018 Grad. %|2019 Grad. %|
+---------+--------------------+-----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   400121|  ACERO - DE LA CRUZ|             null|        78.1|        79.5|        null|        null|        null|        null|        null|        null|        null|
|   400085|   ACERO - GARCIA HS|             null|        75.5|        85.1|        89.5|        84.5|        89.2|        86.3|        87.3|        null|        null|
|   400149|     ACERO - SOTO HS|             null|        83.9|        71.4|        null|        null|        null|        null|        null|        null|      

In [ ]:
grad_and_dropout_2_df = grad_and_dropout_2_df.toPandas()
grad_and_dropout_2_df.head()

,School ID,School Name,Status as of 2019,2011 Grad. %,2012 Grad. %,2013 Grad. %,2014 Grad. %,2015 Grad. %,2016 Grad. %,2017 Grad. %,2018 Grad. %,2019 Grad. %
0,400121,ACERO - DE LA CRUZ,None,78.1,79.5,None,None,None,None,None,None,None
1,400085,ACERO - GARCIA HS,None,75.5,85.1,89.5,84.5,89.2,86.3,87.3,None,None
2,400149,ACERO - SOTO HS,None,83.9,71.4,None,None,None,None,None,None,None
3,610513,AIR FORCE HS,None,70.2,84.0,86.1,86.7,81.9,83.7,None,None,None
4,610524,ALCOTT HS,None,86.3,77.7,87.1,81.3,84.9,92.1,None,None,None


In [ ]:
url10 = "https://cps-final-project-bucket.s3.us-east-2.amazonaws.com/metrics_attendance_clean3.csv"
spark.sparkContext.addFile(url10)
attendance2_df = spark.read.csv(SparkFiles.get("metrics_attendance_clean3.csv"), sep=",", header=True)

In [ ]:
attendance2_df = attendance2_df.toPandas()
attendance2_df.head()

,School ID,School Name,Network,Group,Grade,2015,2016,2017,2018,2019
0,609771,ABBOTT,None,Grade,8,None,None,None,None,None
1,400010,ACE TECH HS,None,Grade,8,None,None,None,None,None
2,400153,ACERO - BRIGHTON PARK,Charter,Grade,8,95.3,93.6,95.3,95.8,95.3
3,400101,ACERO - CISNEROS,Charter,Grade,8,96.9,96.7,96.6,95.7,96.2
4,400120,ACERO - CLEMENTE,Charter,Grade,8,95.3,94.8,95.0,95.6,95.6


In [ ]:
url6 = "https://cps-final-project-bucket.s3.us-east-2.amazonaws.com/metrics_fot_schoollevel_2022-1.csv"
spark.sparkContext.addFile(url6)
fot_df = spark.read.csv(SparkFiles.get("metrics_fot_schoollevel_2022-1.csv"), sep=",", header=True)

In [ ]:
fot_df = fot_df.toPandas()
fot_df.head()

,School ID,School Name,SY 2019 On-Track Rate,SY 2019 Total Number of Freshmen,SY 2018 On-Track Rate,SY 2018 Total Number of Freshmen,SY 2017 On-Track Rate,SY 2017 Total Number of Freshmen,SY 2016 On-Track Rate,SY 2016 Total Number of Freshmen,SY 2015 On-Track Rate,SY 2015 Total Number of Freshmen
0,610513,AIR FORCE HS,88.9,54,90.6,32,87.4,95,80.2,101,90.4,104
1,610524,ALCOTT HS,90.0,100,90.7,86,77.6,98,85.3,95,86.7,83
2,609695,AMUNDSEN HS,92.4,369,92.4,327,90.0,309,94.4,322,89.6,278
3,400018,AUSTIN BUS & ENTRP HS,None,None,None,None,None,None,71.4,14,81.6,38
4,610518,AUSTIN CCA HS,87.9,58,76.9,52,70.0,50,55.6,18,68.3,41


In [ ]:
fot_df = fot_df.set_index(['School ID'])
fot_df

,School Name,SY 2019 On-Track Rate,SY 2019 Total Number of Freshmen,SY 2018 On-Track Rate,SY 2018 Total Number of Freshmen,SY 2017 On-Track Rate,SY 2017 Total Number of Freshmen,SY 2016 On-Track Rate,SY 2016 Total Number of Freshmen,SY 2015 On-Track Rate,SY 2015 Total Number of Freshmen
School ID,,,,,,,,,,,
610513,AIR FORCE HS,88.9,54,90.6,32,87.4,95,80.2,101,90.4,104
610524,ALCOTT HS,90.0,100,90.7,86,77.6,98,85.3,95,86.7,83
609695,AMUNDSEN HS,92.4,369,92.4,327,90.0,309,94.4,322,89.6,278
400018,AUSTIN BUS & ENTRP HS,None,None,None,None,None,None,71.4,14,81.6,38
610518,AUSTIN CCA HS,87.9,58,76.9,52,70.0,50,55.6,18,68.3,41
...,...,...,...,...,...,...,...,...,...,...,...
609740,WELLS HS,88.9,90,87.1,124,90.9,88,85.3,68,76.4,144
609693,WESTINGHOUSE HS,97.7,299,98.2,332,96.4,304,95.0,302,94.5,292
610380,WILLIAMS HS,70.0,30,80.9,68,85.4,48,83.1,71,81.1,90


In [ ]:
grad_rate_merge_fot_df = pd.merge(grad_and_dropout_2_df, fot_df, on=['School ID'])
grad_rate_merge_fot_df.head()

,School ID,School Name_x,Status as of 2019,2011 Grad. %,2012 Grad. %,2013 Grad. %,2014 Grad. %,2015 Grad. %,2016 Grad. %,2017 Grad. %,...,SY 2019 On-Track Rate,SY 2019 Total Number of Freshmen,SY 2018 On-Track Rate,SY 2018 Total Number of Freshmen,SY 2017 On-Track Rate,SY 2017 Total Number of Freshmen,SY 2016 On-Track Rate,SY 2016 Total Number of Freshmen,SY 2015 On-Track Rate,SY 2015 Total Number of Freshmen
0,610513,AIR FORCE HS,None,70.2,84.0,86.1,86.7,81.9,83.7,None,...,88.9,54,90.6,32,87.4,95,80.2,101,90.4,104
1,610524,ALCOTT HS,None,86.3,77.7,87.1,81.3,84.9,92.1,None,...,90.0,100,90.7,86,77.6,98,85.3,95,86.7,83
2,609695,AMUNDSEN HS,None,63.8,64.7,77.1,74.7,75.8,82.8,76.0,...,92.4,369,92.4,327,90.0,309,94.4,322,89.6,278
3,610518,AUSTIN CCA HS,None,67.8,55.4,70.9,70.9,77.3,77.6,64.7,...,87.9,58,76.9,52,70.0,50,55.6,18,68.3,41
4,610563,BACK OF THE YARDS HS,None,91.8,97.3,None,None,None,None,None,...,97.5,276,100.0,274,99.6,275,100.0,261,100.0,275


In [ ]:
grad_rate_merge_fot_df = grad_rate_merge_fot_df[grad_rate_merge_fot_df['2016 Grad. %'].notnull()]
grad_rate_merge_fot_df

,School ID,School Name_x,Status as of 2019,2011 Grad. %,2012 Grad. %,2013 Grad. %,2014 Grad. %,2015 Grad. %,2016 Grad. %,2017 Grad. %,...,SY 2019 On-Track Rate,SY 2019 Total Number of Freshmen,SY 2018 On-Track Rate,SY 2018 Total Number of Freshmen,SY 2017 On-Track Rate,SY 2017 Total Number of Freshmen,SY 2016 On-Track Rate,SY 2016 Total Number of Freshmen,SY 2015 On-Track Rate,SY 2015 Total Number of Freshmen
0,610513,AIR FORCE HS,None,70.2,84.0,86.1,86.7,81.9,83.7,None,...,88.9,54,90.6,32,87.4,95,80.2,101,90.4,104
1,610524,ALCOTT HS,None,86.3,77.7,87.1,81.3,84.9,92.1,None,...,90.0,100,90.7,86,77.6,98,85.3,95,86.7,83
2,609695,AMUNDSEN HS,None,63.8,64.7,77.1,74.7,75.8,82.8,76.0,...,92.4,369,92.4,327,90.0,309,94.4,322,89.6,278
3,610518,AUSTIN CCA HS,None,67.8,55.4,70.9,70.9,77.3,77.6,64.7,...,87.9,58,76.9,52,70.0,50,55.6,18,68.3,41
5,609698,BOGAN HS,None,52.0,53.6,60.3,66.8,74.3,75.7,71.4,...,86.5,222,95.2,189,97.2,177,92.9,198,89.4,189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,610392,WORLD LANGUAGE HS,None,67.8,59.8,82.5,69.8,82.3,77.9,85.0,...,84.4,96,93.5,93,93.8,96,86.6,82,84.0,75
88,609755,YOUNG HS,None,93.9,91.1,94.4,93.3,91.4,96.2,96.1,...,98.6,498,98.4,516,98.3,475,97.9,433,99.1,448
89,400018,AUSTIN BUS & ENTRP HS,Closed,65.6,53.8,60.9,50.0,63.5,76.5,71.0,...,None,None,None,None,None,None,71.4,14,81.6,38
90,610501,AUSTIN POLY HS,Closed,63.1,57.1,60.3,50.7,60.9,47.1,55.6,...,None,None,None,None,None,None,66.7,21,80.6,31


In [ ]:
persist_merge_fot_df = pd.merge(persist_3_df, fot_df, on=['School ID'])
persist_merge_fot_df.head()

,Annualized School Name,status,Class of 2019 Graduates,Class of 2019 Enrollments,Class of 2019 Enrollment Pct,Class of 2019 # of Enrollments Persisting,Class of 2019 Persistence Pct,Class of 2018 Graduates,Class of 2018 Enrollments,Class of 2018 Enrollment Pct,...,SY 2019 On-Track Rate,SY 2019 Total Number of Freshmen,SY 2018 On-Track Rate,SY 2018 Total Number of Freshmen,SY 2017 On-Track Rate,SY 2017 Total Number of Freshmen,SY 2016 On-Track Rate,SY 2016 Total Number of Freshmen,SY 2015 On-Track Rate,SY 2015 Total Number of Freshmen
School ID,,,,,,,,,,,,,,,,,,,,,
609674,CHICAGO VOCATIONAL HS,None,200,97,48.5,42,43.3,210,80,38.1,...,91.5,260,88.8,206,89.7,174,92.0,224,85.2,243
609676,DUNBAR HS,None,76,53,69.7,29,54.7,99,57,57.6,...,90.3,62,81.4,86,78.4,102,75.5,147,62.4,186
609678,JONES HS,None,435,383,88,352,91.9,463,400,86.4,...,98.0,509,99.6,452,99.1,423,99.1,428,99.0,482
609679,PROSSER HS,None,298,211,70.8,133,63,293,224,76.5,...,82.5,275,92.4,397,95.8,355,94.7,356,86.6,357
609680,PAYTON HS,None,213,198,93,184,92.9,219,202,92.2,...,99.7,310,98.8,336,99.3,294,99.6,229,96.9,229


## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [ ]:
grad_rate_merge_fot_df.dtypes

School ID                           object
School Name_x                       object
Status as of 2019                   object
2011 Grad. %                        object
2012 Grad. %                        object
2013 Grad. %                        object
2014 Grad. %                        object
2015 Grad. %                        object
2016 Grad. %                        object
2017 Grad. %                        object
2018 Grad. %                        object
2019 Grad. %                        object
School Name_y                       object
SY 2019 On-Track Rate               object
SY 2019 Total Number of Freshmen    object
SY 2018 On-Track Rate               object
SY 2018 Total Number of Freshmen    object
SY 2017 On-Track Rate               object
SY 2017 Total Number of Freshmen    object
SY 2016 On-Track Rate               object
SY 2016 Total Number of Freshmen    object
SY 2015 On-Track Rate               object
SY 2015 Total Number of Freshmen    object
dtype: obje

In [ ]:
grad_rate_merge_fot_df['SY 2016 On-Track Rate'].astype(float)

0     80.2
1     85.3
2     94.4
3     55.6
5     92.9
      ... 
87    86.6
88    97.9
89    71.4
90    66.7
91    57.9
Name: SY 2016 On-Track Rate, Length: 84, dtype: float64

In [ ]:
grad_rate_merge_fot_df['2016 Grad. %'].astype(float)

0     83.7
1     92.1
2     82.8
3     77.6
5     75.7
      ... 
87    77.9
88    96.2
89    76.5
90    47.1
91    39.4
Name: 2016 Grad. %, Length: 84, dtype: float64

In [ ]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = grad_rate_merge_fot_df['SY 2016 On-Track Rate']

# Separate the X variable, the features
X = grad_rate_merge_fot_df['2016 Grad. %']

In [ ]:
# Review the y variable Series
y.head()

0    80.2
1    85.3
2    94.4
3    55.6
5    92.9
Name: SY 2016 On-Track Rate, dtype: object

In [ ]:
# Review the X variable DataFrame
X.head()

0    83.7
1    92.1
2    82.8
3    77.6
5    75.7
Name: 2016 Grad. %, dtype: object

### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [ ]:
# Check the balance of our target values
y.value_counts()

71.4    3
99.3    2
83.0    2
83.1    2
85.1    2
       ..
74.4    1
75.3    1
90.1    1
84.1    1
57.9    1
Name: SY 2016 On-Track Rate, Length: 71, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
X_train.keys()

Int64Index([ 2, 89, 40, 54, 16, 77, 58, 22, 49, 84, 78, 31, 90, 51, 69, 44, 24,
            81,  3, 57, 63, 52, 47, 60, 29,  9, 19,  0, 87, 64,  8, 88, 66, 14,
            25, 37, 35, 61, 55, 46, 68, 26,  5, 82, 15, 34, 33, 12, 21, 23, 56,
            30,  7, 79,  1, 17, 70,  6, 83, 10, 80, 13, 42],
           dtype='int64')

In [ ]:
X_train.values

array(['82.8', '76.5', '77.8', '80.9', '61.0', '85.2', '94.5', '71.5',
       '57.9', '62.9', '75.9', '62.1', '47.1', '57.7', '81.3', '89.4',
       '49.5', '27.8', '77.6', '97.8', '88.2', '72.3', '87.7', '95.3',
       '84.6', '91.0', '76.0', '83.7', '77.9', '73.4', '93.4', '96.2',
       '87.6', '85.8', '65.2', '91.8', '53.6', '57.9', '74.3', '95.9',
       '72.5', '68.7', '75.7', '88.8', '83.5', '82.9', '71.4', '85.4',
       '56.3', '65.3', '3.1', '54.6', '80.6', '50.0', '92.1', '62.6',
       '80.7', '63.7', '72.2', '85.4', '74.3', '72.6', '68.8'],
      dtype=object)

---

In [ ]:
X_train = X_train.to_numpy()

In [ ]:
X_train_reshape = X_train.reshape(1,-1)

AttributeError: ignored

In [ ]:
X_train

array(['82.8', '76.5', '77.8', '80.9', '61.0', '85.2', '94.5', '71.5',
       '57.9', '62.9', '75.9', '62.1', '47.1', '57.7', '81.3', '89.4',
       '49.5', '27.8', '77.6', '97.8', '88.2', '72.3', '87.7', '95.3',
       '84.6', '91.0', '76.0', '83.7', '77.9', '73.4', '93.4', '96.2',
       '87.6', '85.8', '65.2', '91.8', '53.6', '57.9', '74.3', '95.9',
       '72.5', '68.7', '75.7', '88.8', '83.5', '82.9', '71.4', '85.4',
       '56.3', '65.3', '3.1', '54.6', '80.6', '50.0', '92.1', '62.6',
       '80.7', '63.7', '72.2', '85.4', '74.3', '72.6', '68.8'],
      dtype=object)

In [ ]:
X_train = X_train.tolist()
X_train


['82.8',
 '76.5',
 '77.8',
 '80.9',
 '61.0',
 '85.2',
 '94.5',
 '71.5',
 '57.9',
 '62.9',
 '75.9',
 '62.1',
 '47.1',
 '57.7',
 '81.3',
 '89.4',
 '49.5',
 '27.8',
 '77.6',
 '97.8',
 '88.2',
 '72.3',
 '87.7',
 '95.3',
 '84.6',
 '91.0',
 '76.0',
 '83.7',
 '77.9',
 '73.4',
 '93.4',
 '96.2',
 '87.6',
 '85.8',
 '65.2',
 '91.8',
 '53.6',
 '57.9',
 '74.3',
 '95.9',
 '72.5',
 '68.7',
 '75.7',
 '88.8',
 '83.5',
 '82.9',
 '71.4',
 '85.4',
 '56.3',
 '65.3',
 '3.1',
 '54.6',
 '80.6',
 '50.0',
 '92.1',
 '62.6',
 '80.7',
 '63.7',
 '72.2',
 '85.4',
 '74.3',
 '72.6',
 '68.8']

In [ ]:
X_train_reshape

array([['82.8', '76.5', '77.8', '80.9', '61.0', '85.2', '94.5', '71.5',
        '57.9', '62.9', '75.9', '62.1', '47.1', '57.7', '81.3', '89.4',
        '49.5', '27.8', '77.6', '97.8', '88.2', '72.3', '87.7', '95.3',
        '84.6', '91.0', '76.0', '83.7', '77.9', '73.4', '93.4', '96.2',
        '87.6', '85.8', '65.2', '91.8', '53.6', '57.9', '74.3', '95.9',
        '72.5', '68.7', '75.7', '88.8', '83.5', '82.9', '71.4', '85.4',
        '56.3', '65.3', '3.1', '54.6', '80.6', '50.0', '92.1', '62.6',
        '80.7', '63.7', '72.2', '85.4', '74.3', '72.6', '68.8']],
      dtype=object)

In [ ]:
y_train = y_train.to_numpy()


AttributeError: ignored

In [ ]:
y_train

array(['94.4', '71.4', '74.4', '77.1', '69.7', '86.0', '93.6', '75.5',
       '72.1', '85.3', '60.0', '76.4', '66.7', '93.2', '92.8', '90.1',
       '93.9', '95.2', '55.6', '99.3', '94.7', '76.5', '87.7', '99.6',
       '98.5', '96.6', '71.7', '80.2', '86.6', '83.6', '94.4', '97.9',
       '95.8', '91.2', '68.7', '99.1', '86.3', '73.5', '91.4', '97.7',
       '78.3', '83.1', '92.9', '87.2', '93.3', '82.3', '84.4', '96.4',
       '77.8', '85.1', '100.0', '58.1', '96.4', '70.8', '85.3', '88.4',
       '81.7', '92.4', '93.3', '83.0', '85.1', '92.0', '83.3'],
      dtype=object)

In [ ]:
len(y_train)

63

In [ ]:
len(X_train_reshape)

1

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [ ]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train, y_train)

ValueError: ignored

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [ ]:
# Make a prediction using the testing data
test_predictions = logistic_regression_model.predict(X_test)
pd.DataFrame({'Predictions': test_predictions, 'Actual': y_test})

,Predictions,Actual
60914,0,0
36843,0,0
1966,0,0
70137,0,0
27237,0,0
...,...,...
45639,0,0
11301,0,0
51614,0,0
4598,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
# Print the balanced_accuracy score of the model
print(f"The balanced accuracy score of the model is: {balanced_accuracy_score(y_test, test_predictions)}")

The balanced accuracy score of the model is: 0.9520479254722232


In [ ]:
# Generate a confusion matrix for the model
cf_test_matrix = confusion_matrix(y_test, test_predictions)
cf_test_matrix

array([[18663,   102],
       [   56,   563]], dtype=int64)

In [ ]:
# Print the classification report for the model
testing_report = classification_report(y_test, test_predictions)
print(testing_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model was 95% accurate at predicting the healthy vs high-risk loan labels

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [ ]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_ros_model, y_ros_model = ros.fit_resample(X,y)

In [ ]:
from collections import Counter
# Count the distinct values of the resampled labels data
print(Counter(X_ros_model))
print(Counter(y_ros_model))
print(f"The y_ros_model resampled data is equivalently split")

Counter({'loan_size': 1, 'interest_rate': 1, 'borrower_income': 1, 'debt_to_income': 1, 'num_of_accounts': 1, 'derogatory_marks': 1, 'total_debt': 1})
Counter({0: 75036, 1: 75036})
The y_ros_model resampled data is equivalently split


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [ ]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier.fit(X_ros_model, y_ros_model)
# Make a prediction using the testing data
predictions = classifier.predict(X_ros_model)
pd.DataFrame({'Predictions': predictions, 'Actual': y_ros_model})


,Predictions,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
150067,1,1
150068,1,1
150069,1,1
150070,1,1


In [ ]:
#####DELETE######
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier.fit(X_train, y_train)
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({'Predictions': predictions, 'Actual': y_test})

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
# Print the balanced_accuracy score of the model
print(f"The balanced accuracy score of the model is: {balanced_accuracy_score(y_ros_model, predictions)}")

The balanced accuracy score of the model is: 0.9945026387334079


In [ ]:
# Generate a confusion matrix for the model
cf_matrix = confusion_matrix(y_ros_model, predictions)
cf_matrix

array([[74614,   422],
       [  403, 74633]], dtype=int64)

In [ ]:
# Print the classification report for the model
report = classification_report(y_ros_model, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     75036
           1       0.99      0.99      0.99     75036

    accuracy                           0.99    150072
   macro avg       0.99      0.99      0.99    150072
weighted avg       0.99      0.99      0.99    150072



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicts the oversampled data with near-perfect accuracy (>99% accurate)